In [1]:
import cv2
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

In [2]:
import os

blocked_dir = 'dataset/blocked'
free_dir = 'dataset/free'
# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created because they already exist')

In [7]:
# Button layout
button_layout = widgets.Layout(width='128px', height='64px')
camera = cv2.VideoCapture(0)
width, height = 224, 224

# Create a widgets.Image instance for displaying the video stream
image_widget = widgets.Image(format='jpeg', width=width, height=height)

# Display the image widget
display(image_widget)

Image(value=b'', format='jpeg', height='224', width='224')

In [8]:

# Button callbacks
def add_free_button_clicked(b):
    global free_count
    # Capture an image from the camera
    ret, frame = camera.read()
    if ret:
        rotated_frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        rotated_frame = cv2.rotate(rotated_frame, cv2.ROTATE_90_CLOCKWISE)
        
        # Resize the rotated frame
        resized_frame = cv2.resize(rotated_frame, (width, height))
        # Save the image to the free directory
        image_filename = os.path.join(free_dir, f'free_{free_count.value}.jpg')
        cv2.imwrite(image_filename, resized_frame)
        free_count.value += 1
        update_image_widget(1)


def add_blocked_button_clicked(b):
    global blocked_count
    # Capture an image from the camera
    ret, frame = camera.read()
    if ret:
        rotated_frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        rotated_frame = cv2.rotate(rotated_frame, cv2.ROTATE_90_CLOCKWISE)
        
        # Resize the rotated frame
        resized_frame = cv2.resize(rotated_frame, (width, height))
        # Save the image to the blocked directory
        image_filename = os.path.join(blocked_dir, f'blocked_{blocked_count.value}.jpg')
        cv2.imwrite(image_filename, resized_frame)
        blocked_count.value += 1
        update_image_widget(1)

# Create buttons
free_button = widgets.Button(description='add free', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='add blocked', button_style='danger', layout=button_layout)

# Set button callbacks
free_button.on_click(add_free_button_clicked)
blocked_button.on_click(add_blocked_button_clicked)

# Get initial counts
initial_free_count = len(os.listdir(free_dir))
initial_blocked_count = len(os.listdir(blocked_dir))

# Create count widgets
free_count = widgets.IntText(layout=button_layout, value=initial_free_count)
blocked_count = widgets.IntText(layout=button_layout, value=initial_blocked_count)

# Display widgets
display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))


In [9]:
def update_image_widget(b):
    # Capture an image from the camera
    ret, frame = camera.read()
    if ret:
        rotated_frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        rotated_frame = cv2.rotate(rotated_frame, cv2.ROTATE_90_CLOCKWISE)

        # Resize the frame
        resized_frame = cv2.resize(rotated_frame, (width, height))
        # Convert BGR to RGB (OpenCV uses BGR)
        rgb_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
        # Convert to JPEG format
        _, jpeg_frame = cv2.imencode('.jpeg', rgb_frame)
        # Update the image widget
        image_widget.value = jpeg_frame.tobytes()

# Display the image widget
display(image_widget)

# Display the free and blocked count widgets and buttons
display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

# Add a button to update the image widget
update_button = widgets.Button(description='Update Image', button_style='info', layout=button_layout)
update_button.on_click(update_image_widget)
display(update_button)


Image(value=b'', format='jpeg', height='224', width='224')

Button(button_style='info', description='Update Image', layout=Layout(height='64px', width='128px'), style=But…

In [11]:
camera.release()